# Experiment Notebook: Mento 1.0 Model Validation

# Table of Contents
* [Experiment Summary](#Experiment-Summary)
* [Experiment Assumptions](#Experiment-Assumptions)
* [Experiment Setup](#Experiment-Setup)
* [Analysis 1: Profit Yields Across Validator Environments](#Analysis-1:-Profit-Yields-Across-Validator-Environments)
* [Analysis 2: Network Yields and Network Inflation](#Analysis-2:-Network-Yields-and-Network-Inflation)
* [Analysis 3: Revenue/Profit Yield Spread ("Three Region Analysis")](#Analysis-3:-Revenue/Profit-Yield-Spread-("Three-Region-Analysis"))

# Experiment Summary 

The purpose of this notebook is to analyze the Mento 1.0 setup and test and validate the Mento 1.0 model by backchecking the results of this analysis against empiracal Mento 1.0 data.

# Experiment Assumptions

Our model adopts a range of assumptions... tbd

See [assumptions document](../../ASSUMPTIONS.md) for further details.

# Experiment Setup

We begin with several experiment-notebook-level preparatory setup operations:

* Import relevant dependencies
* Import relevant experiment templates
* Create copies of experiments
* Configure and customize experiments 

Analysis-specific setup operations are handled in their respective notebook sections.

In [ ]:
import setup

import copy
import logging
import numpy as np
from radcad import Engine

import experiments.notebooks.visualizations as visualizations
from experiments.run import run
from model.types import Stage

In [ ]:
# Enable/disable logging
logger = logging.getLogger()
logger.disabled = True

In [ ]:
# Import experiment templates
from experiments.default_experiment import experiment, TIMESTEPS, DELTA_TIME

In [ ]:
# Create a copy of the simulation
simulation = copy.deepcopy(experiment.simulations[0])

# Override default-experiment System Parameters
# using assumptions from Hoban/Borgers Economic Report
simulation.model.params.update({
    "stage": [Stage.BEACON_CHAIN],
    "daily_pow_issuance": [0],  # ETH
    # Combine the validator internet (99.9%), power (99.9%), and technical (98.2%) uptime
    # from Hoban/Borgers Report
    "validator_uptime_process": [lambda _run, _timestep: 0.999 * 0.999 * 0.982],  # Percentage (unitless)
})

# Override default-experiment State Variables
# using assumptions from Hoban/Borgers Economic Report
simulation.model.initial_state.update({
    "eth_supply": 112_000_000,  # ETH
    "eth_price": 25,  # USD/ETH
    "eth_staked": 524_288,  # ETH
    "number_of_validators": 16_384,  # Unitless
})

# Set runs to number of ETH price / staked samples
simulation.runs = 50
# Run single timestep, set unit of time `dt` to multiple epochs
# (see 0_README.ipynb for further details)
simulation.timesteps = 1
simulation.model.params.update({"dt": [TIMESTEPS * DELTA_TIME]})

# Drop state history substeps to improve performance
# (see 0_README.ipynb for further details)
simulation.engine = Engine(drop_substeps=True)

### Configuration

In [ ]:
# Create a copy of the simulation
simulation_1 = copy.deepcopy(simulation)

# Create a range of 50 discrete ETH price values starting at
# 25 USD/ETH and ending at 1500 USD/ETH
# Assumption adopted from Hoban/Borgers Economic Report
eth_price_samples = np.linspace(
    start=25,
    stop=1500,
    num=50
)

parameter_overrides = {
    "eth_price_process": [
        # Sample the ETH price values using the run as the index
        lambda run, timestep: eth_price_samples[run - 1]
    ],
    "eth_staked_process": [
        # A sweep of two fixed ETH staked points
        # Assumption adopted from Hoban/Borgers Economic Report
        lambda _run, _timestep: 524_288,  # Beacon Chain genesis ETH staked requirement
        lambda _run, _timestep: 33_600_000,  # 30% of the total ETH supply
    ],
}

# Override default experiment parameters
simulation_1.model.params.update(parameter_overrides)

### Execution

In [ ]:
df_1, exceptions = run(simulation_1)

### Output Preparation

In [ ]:
df_1